In [54]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import nltk

from datasets import load_dataset
trainingdata = pd.read_csv('Data/Train.csv')
testdata = pd.read_csv('Data/Test.csv')
validationdata = pd.read_csv('Data/Valid.csv')

trainingdata.describe()

,label
count,40000.000000
mean,0.499525
std,0.500006
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [55]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [56]:

def polarity_scores_roberta(input):
    encoded_text = tokenizer(input, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scores_dict


result = {}
for i, row in testdata.iterrows():
    try:
        result[i] = polarity_scores_roberta(row['text'])
    except RuntimeError:
        result[i] = {
        'roberta_neg' : 0,
        'roberta_neu' : 0,
        'roberta_pos' : 0
    }
    except IndexError:
        result[i] = {
        'roberta_neg' : 0,
        'roberta_neu' : 0,
        'roberta_pos' : 0
    }
roberta = pd.DataFrame(result).T
roberta = roberta.join(testdata, how='left')
roberta.to_csv('Data/out.csv')


In [57]:
roberta = pd.DataFrame(result).T
print(result)
roberta = roberta.join(testdata, how='left')
roberta.to_csv('Data/out.csv')

{0: {'roberta_neg': 0.16739884, 'roberta_neu': 0.43455082, 'roberta_pos': 0.39805034}, 1: {'roberta_neg': 0.53142095, 'roberta_neu': 0.38358888, 'roberta_pos': 0.084990114}, 2: {'roberta_neg': 0.918127, 'roberta_neu': 0.07046218, 'roberta_pos': 0.011410812}, 3: {'roberta_neg': 0, 'roberta_neu': 0, 'roberta_pos': 0}, 4: {'roberta_neg': 0.7044716, 'roberta_neu': 0.22220136, 'roberta_pos': 0.073327124}, 5: {'roberta_neg': 0.033169143, 'roberta_neu': 0.33361632, 'roberta_pos': 0.63321453}, 6: {'roberta_neg': 0.40910995, 'roberta_neu': 0.37812874, 'roberta_pos': 0.21276128}, 7: {'roberta_neg': 0.090338044, 'roberta_neu': 0.42419025, 'roberta_pos': 0.48547176}, 8: {'roberta_neg': 0.8070362, 'roberta_neu': 0.15814832, 'roberta_pos': 0.034815487}, 9: {'roberta_neg': 0.9455735, 'roberta_neu': 0.047612168, 'roberta_pos': 0.006814316}, 10: {'roberta_neg': 0, 'roberta_neu': 0, 'roberta_pos': 0}, 11: {'roberta_neg': 0.037293095, 'roberta_neu': 0.16991681, 'roberta_pos': 0.7927901}, 12: {'roberta_ne

In [58]:
print(polarity_scores_roberta("I am feeling stressed over college this morning"))
print(polarity_scores_roberta("I am feeling anxious over college this morning"))
print(polarity_scores_roberta("I am feeling depressed over college this morning"))

{'roberta_neg': 0.8847369, 'roberta_neu': 0.10381528, 'roberta_pos': 0.011447806}
{'roberta_neg': 0.853196, 'roberta_neu': 0.13171409, 'roberta_pos': 0.015089901}
{'roberta_neg': 0.96005565, 'roberta_neu': 0.035580818, 'roberta_pos': 0.004363502}


: 